Testing on small amount of data first

In [13]:
# Sets the path to the terrain data directory
DATA_DIR = r'C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\Assets\Snapshots\terrain_data'

In [7]:
# This was for setting up the environment required for training the model
# %cd "C:\Users\User\OneDrive\Documents\GitHub\image-classification-yolov8"
# %pip install -r requirements.txt

C:\Users\User\OneDrive\Documents\GitHub\image-classification-yolov8


In [2]:
import os 

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n-cls.pt")  # load a pretained model

# Use the model
results = model.train(data=DATA_DIR, epochs=10, imgsz=64)  # train the model


100%|██████████| 5.28M/5.28M [00:01<00:00, 4.04MB/s]
New https://pypi.org/project/ultralytics/8.0.221 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.58  Python-3.11.6 torch-2.1.1+cpu CPU
yolo\engine\trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\Assets\Snapshots\terrain_data, epochs=10, patience=50, batch=16, imgsz=64, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_strid

In [14]:
# now set the new model path to the best model 
model_path = r'C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\ModelTraining\runs\detect\terrain_train\runs\classify\train\weights\best.pt'
# create a separate variable for the path to the test data set 
test_data_path = r'C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\Assets\Snapshots\terrain_test_spec'

In [12]:
# Run val on the test data :) to get the accuracy of the model (once you've swapped in the folder whe)
test_model = YOLO(model_path)   

test_model.val(test_data_path)

Ultralytics YOLOv8.0.58  Python-3.11.6 torch-2.1.1+cpu CPU
YOLOv8n-cls summary (fused): 73 layers, 1438723 parameters, 0 gradients, 3.3 GFLOPs
               classes   top1_acc   top5_acc: 100%|██████████| 2/2 [00:00<00:00,  6.61it/s]
                   all      0.926          1
Speed: 0.0ms preprocess, 2.8ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs\classify\val2


ultralytics.yolo.utils.metrics.ClassifyMetrics object with attributes:

fitness: 1.0
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9259259104728699, 'metrics/accuracy_top5': 1.0, 'fitness': 1.0}
speed: {'preprocess': 0.0, 'inference': 2.793320903071651, 'loss': 0.0, 'postprocess': 0.0}
top1: 0.9259259104728699
top5: 1.0

# Test on Larger amount of data

### Delete .meta files

In [5]:
import os

# Remove all .meta files from a directory
def remove_meta_files(directory):
    if not os.path.exists(directory):
        print(f"The directory {directory} does not exist.")
        return

    for filename in os.listdir(directory):
        if filename.endswith('.meta'):
            file_path = os.path.join(directory, filename)
            os.remove(file_path)
            print(f"Removed {file_path}")

In [ ]:
directory_path = r'C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\Assets\Snapshots\2'
remove_meta_files(directory_path)

### Split Data

Set as 80% training, 10% validation, 10% test


In [7]:
import os
import shutil
import random

def split_data(source_dir, train_dir, val_dir, test_dir, split_ratio=(0.8, 0.1, 0.1)):
    # Ensure the source directory exists
    if not os.path.isdir(source_dir):
        print(f"Source directory {source_dir} does not exist.")
        return

    # Create the destination directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Get all files in the source directory
    all_files = os.listdir(source_dir)
    random.shuffle(all_files)  # Shuffle to randomize selection

    # Calculate split indices
    total_files = len(all_files)
    train_end = int(total_files * split_ratio[0])
    val_end = train_end + int(total_files * split_ratio[1])

    # Split files into train, val, and test
    train_files = all_files[:train_end]
    val_files = all_files[train_end:val_end]
    test_files = all_files[val_end:]

    # Function to copy files to a target directory
    def copy_files(files, target_dir):
        os.makedirs(target_dir, exist_ok=True)
        for file in files:
            shutil.copy(os.path.join(source_dir, file), os.path.join(target_dir, file))

    # Copy files to respective directories
    copy_files(train_files, os.path.join(train_dir, os.path.basename(source_dir)))
    copy_files(val_files, os.path.join(val_dir, os.path.basename(source_dir)))
    copy_files(test_files, os.path.join(test_dir, os.path.basename(source_dir)))

# Directories
base_dir = r'C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\Assets\Snapshots'  
directories = ['0', '1', '2']

for dir in directories:
    split_data(os.path.join(base_dir, dir), 
               os.path.join(base_dir, 'train'), 
               os.path.join(base_dir, 'val'), 
               os.path.join(base_dir, 'test'))

print("Data splitting completed.")


Data splitting completed.


# Training with Lots of Terrain Data


In [ ]:
# Sets the path to the terrain data directory
data_directory = r'C:\Users\User\OneDrive\Documents\GitHub\cv-walking-rlagents\Assets\Snapshots\terrain_data'